In [0]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
bank_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bank.csv')

In [4]:
bank_df.shape

(10000, 14)

In [5]:
bank_df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
bank_df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
bank_df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
list1 = bank_df['RowNumber'].value_counts() > 1
list2 = bank_df['CustomerId'].value_counts() > 1
print(list1.unique())
print(list2.unique())

[False]
[False]


In [0]:
#Row number, CustomerId looks disctinct for each record and is fairly not very useful for our 
#analysis in classifying the customers

#Surname column is not very useful in our analysis and hence we can skip Surname as well

In [0]:
bank_df.drop({'RowNumber','CustomerId', 'Surname'}, axis=1, inplace=True)

In [11]:
bank_df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [0]:
X = bank_df.drop('Exited', axis = 1)
y = bank_df['Exited']

In [13]:
X.shape

(10000, 10)

In [14]:
y.shape

(10000,)

In [15]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
labelEncoder = LabelEncoder()
X['Geography'] = labelEncoder.fit_transform(X['Geography'])
X['Gender'] = labelEncoder.fit_transform(X['Gender'])

In [0]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [0]:
pipeline = Pipeline(
    [('Categorizer', ColumnTransformer(
         [ # Gender
          ("Gender Label encoder", OneHotEncoder(categories='auto', drop='first'), [2]),
           # Geography
          ("Geography One Hot", OneHotEncoder(categories='auto', drop='first'), [1])
         ], remainder='passthrough', n_jobs=1)),
     # Standard Scaler for the classifier
    ('Normalizer', StandardScaler())
      ])

In [0]:
X= pipeline.fit_transform(X)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state = 40)

In [23]:
train_X.shape

(7500, 11)

In [24]:
train_y.shape

(7500,)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [26]:
model = Sequential()
model.add(Dense(6, activation = 'relu', input_shape = (train_X.shape[1], )))
model.add(Dropout(rate=0.1)) 
model.add(Dense(6, activation = 'relu')) 
model.add(Dropout(rate=0.1))
model.add(Dense(1, activation = 'sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy']) #As our dependent variable is Exited with 0 and 1 values we can use binary loss function

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [29]:
history = model.fit(train_X, train_y, batch_size=50, epochs=20, validation_split=0.1, verbose=2)

Train on 6750 samples, validate on 750 samples
Epoch 1/20
6750/6750 - 1s - loss: 0.8047 - acc: 0.5178 - val_loss: 0.6189 - val_acc: 0.7267
Epoch 2/20
6750/6750 - 0s - loss: 0.5951 - acc: 0.7513 - val_loss: 0.5155 - val_acc: 0.8280
Epoch 3/20
6750/6750 - 0s - loss: 0.5335 - acc: 0.7850 - val_loss: 0.4696 - val_acc: 0.8293
Epoch 4/20
6750/6750 - 0s - loss: 0.4998 - acc: 0.7921 - val_loss: 0.4414 - val_acc: 0.8293
Epoch 5/20
6750/6750 - 0s - loss: 0.4802 - acc: 0.7948 - val_loss: 0.4232 - val_acc: 0.8347
Epoch 6/20
6750/6750 - 0s - loss: 0.4696 - acc: 0.7956 - val_loss: 0.4103 - val_acc: 0.8333
Epoch 7/20
6750/6750 - 0s - loss: 0.4575 - acc: 0.7954 - val_loss: 0.4019 - val_acc: 0.8320
Epoch 8/20
6750/6750 - 0s - loss: 0.4527 - acc: 0.7990 - val_loss: 0.3954 - val_acc: 0.8333
Epoch 9/20
6750/6750 - 0s - loss: 0.4478 - acc: 0.7978 - val_loss: 0.3901 - val_acc: 0.8347
Epoch 10/20
6750/6750 - 0s - loss: 0.4413 - acc: 0.8024 - val_loss: 0.3854 - val_acc: 0.8333
Epoch 11/20
6750/6750 - 0s - los

In [30]:
y_pred = model.predict(test_X)
print(y_pred[:5])

[[0.19007012]
 [0.25453305]
 [0.01781863]
 [0.17731044]
 [0.05823484]]


In [32]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [33]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred)
print(cm)

[[1951   60]
 [ 305  184]]


In [0]:
# Optimizing the parameters
model2 = Sequential()
model2.add(Dense(20, activation = 'relu', input_shape = (train_X.shape[1], )))
model2.add(Dropout(rate=0.1)) 
model2.add(Dense(10, activation = 'relu')) 
model2.add(Dropout(rate=0.1))
model2.add(Dense(5, activation = 'relu'))
model2.add(Dropout(rate=0.1))
model2.add(Dense(1, activation = 'sigmoid'))

In [0]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
history2 = model2.fit(train_X, train_y, batch_size=32, epochs=30, validation_split=0.1, verbose=2)

Train on 6750 samples, validate on 750 samples
Epoch 1/30
6750/6750 - 1s - loss: 0.5440 - acc: 0.7658 - val_loss: 0.4202 - val_acc: 0.8253
Epoch 2/30
6750/6750 - 1s - loss: 0.4711 - acc: 0.7944 - val_loss: 0.3949 - val_acc: 0.8333
Epoch 3/30
6750/6750 - 1s - loss: 0.4535 - acc: 0.8015 - val_loss: 0.3850 - val_acc: 0.8440
Epoch 4/30
6750/6750 - 1s - loss: 0.4425 - acc: 0.8081 - val_loss: 0.3770 - val_acc: 0.8480
Epoch 5/30
6750/6750 - 1s - loss: 0.4323 - acc: 0.8153 - val_loss: 0.3636 - val_acc: 0.8600
Epoch 6/30
6750/6750 - 1s - loss: 0.4184 - acc: 0.8234 - val_loss: 0.3465 - val_acc: 0.8627
Epoch 7/30
6750/6750 - 1s - loss: 0.4034 - acc: 0.8308 - val_loss: 0.3341 - val_acc: 0.8733
Epoch 8/30
6750/6750 - 1s - loss: 0.3891 - acc: 0.8375 - val_loss: 0.3238 - val_acc: 0.8773
Epoch 9/30
6750/6750 - 1s - loss: 0.3895 - acc: 0.8335 - val_loss: 0.3233 - val_acc: 0.8747
Epoch 10/30
6750/6750 - 1s - loss: 0.3787 - acc: 0.8422 - val_loss: 0.3147 - val_acc: 0.8813
Epoch 11/30
6750/6750 - 1s - los

In [37]:
y_pred2 = model2.predict(test_X)
print(y_pred2[:5])

[[0.16433296]
 [0.19722253]
 [0.03252789]
 [0.1920329 ]
 [0.04161999]]


In [40]:
y_pred2 = (y_pred2 > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [41]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred2)
print(cm)

[[1907  104]
 [ 245  244]]
